In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from src.utils import (
                       TextGenerationDataset,
                       RougeDataset,
                       RnnTextGenerator,
                       sentense_generation_inference,
                       train_model,
                       evaluate_rouge_gpt,
                       pretty_output,
                       clean_string,)
from src.constants import PATH_DATA, PATH_MODEL

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
random_state=42
data_file_path = f'{PATH_DATA}tweets.txt'
model_revision = 0

# Этап 1. Сбор и подготовка данных

##  1.1 Загрузка данных

In [ ]:
with open(data_file_path, "r", encoding="utf8") as text_file:
    data_raw = text_file.readlines()

In [ ]:
df_raw = pd.DataFrame(data_raw, columns=['tweet'])

In [ ]:
df_raw.to_csv(f'{PATH_DATA}raw_dataset.csv')

In [ ]:
df_raw = pd.read_csv(f'{PATH_DATA}raw_dataset.csv', index_col=0)

## 1.2 Предобработка данных

In [ ]:
df_processed = df_raw['tweet'].apply(lambda x: clean_string(x))

In [ ]:
df_processed = pd.read_csv(f'{PATH_DATA}dataset_processed.csv', index_col=0)

In [ ]:
df_processed.to_csv(f'{PATH_DATA}dataset_processed.csv')

## 1.3 Разбиение на train / valid / test

In [ ]:
df_processed.shape

In [ ]:
train_texts, valtest_texts = train_test_split(df_processed['tweet'][:50000].tolist(), test_size=0.2, random_state=random_state)
val_texts, test_texts = train_test_split(valtest_texts, test_size=0.5, random_state=random_state)

In [ ]:
len(train_texts), len(val_texts), len(test_texts)

In [ ]:
train_texts[-10:]

## 1.4 Создание объектов Dataset и Dataloader для обучения и валидации

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.add_special_tokens({'bos_token': '[BOS]', 'eos_token': '[EOS]'})
print(tokenizer.eos_token, tokenizer.eos_token_id)

In [ ]:
# # Access the vocabulary dictionary
# vocab_dict = tokenizer.vocab
# new_dict = {value:key for (key,value) in vocab_dict.items()}

In [ ]:
# тренировочный и валидационный датасеты
train_dataset = TextGenerationDataset(train_texts, tokenizer,)
val_dataset = TextGenerationDataset(val_texts, tokenizer,)

# даталоадеры
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

## 1.5 Создание объектов Dataset и Dataloader вычисления метрики rouge

In [ ]:
val_rouge_dataset = RougeDataset(val_texts, n=0.75)
val_rouge_dataloader = DataLoader(
    val_rouge_dataset,
    batch_size=64,
    shuffle=True
)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Этап 2. Реализация рекуррентной сети

## 2.1 Создание модели-LSTM

In [ ]:
vocab_size = len(tokenizer)
hidden_dim = 128

model = RnnTextGenerator(vocab_size, hidden_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.002)
criterion = nn.CrossEntropyLoss()

# Этап 3. Тренировка модели

## 3.1 Обучение модели

In [ ]:
train_data = train_model(
    model,
    train_loader,
    val_loader,
    val_rouge_dataloader,
    tokenizer,
    optimizer,
    criterion,
    device,
    n_epochs=5,
    max_train_batches_in_epoch=1000,
    max_evalacc_batches_in_epoch=100,
    max_evalrouge_steps_count=100,)

In [ ]:
train_data

In [ ]:
rouge1_rnn_avg = train_data[-1]['val_rouge1']
rouge2_rnn_avg = train_data[-1]['val_rouge2']
rouge1_rnn_avg, rouge2_rnn_avg

In [ ]:
model_revision += 1

In [ ]:
model_save_path = f'{PATH_MODEL}rnn_model_weights_rev{model_revision}.pth'

In [ ]:
model_save_path

In [ ]:
torch.save(model.state_dict(), model_save_path)

In [ ]:
# model_loaded = RnnTextGenerator(vocab_size, hidden_dim)  # архитектура должна совпадать с сохранённой
# model_loaded.load_state_dict(torch.load(model_save_path))

# Этап 4. Использование предобученного трансформера

## 4.1 Создание модели-трансформера

In [ ]:
model_gpt2_name = "distilgpt2"          # лёгкая версия GPT-2
tokenizer_gpt2 = AutoTokenizer.from_pretrained(model_gpt2_name)
model_gpt2 = AutoModelForCausalLM.from_pretrained(model_gpt2_name).to(device)

In [ ]:
generator = pipeline(
    task="text-generation",
    model=model_gpt2,
    tokenizer=tokenizer_gpt2,
    device=device  # -1 = CPU; 0 = первый GPU (если есть)
)

## 4.2 Расчет метрики rouge на валидационном датасете

In [ ]:
rouge1_gpt2_avg, rouge2_gpt2_avg = evaluate_rouge_gpt(generator, val_rouge_dataloader, max_steps_cnt=10)

In [ ]:
rouge1_gpt2_avg, rouge2_gpt2_avg

In [ ]:
df_rouge = pd.DataFrame(
    [[rouge1_rnn_avg, rouge2_rnn_avg], [rouge1_gpt2_avg, rouge2_gpt2_avg]],
    columns=['rouge1', 'rouge2'],
    index=['RNN', 'GPT2']
)

## 4.3 Выведем некоторые результаты

In [ ]:
x_texts, y_texts = next(iter(val_rouge_dataloader))

data = []

for i, (x_text, y_text) in enumerate(zip(x_texts, y_texts)):
  if i > 20:
    break

  rnn_generated = sentense_generation_inference(
        x_text,
        model,
        tokenizer,
        device,
        min_input_seq_len=15,
        max_total_length=128,
    )

  gpt_generated = generator(
                x_text,
                max_length=80,       # итоговая длина (включая prompt)
                num_return_sequences=1,
                do_sample=True,      # стохастическая генерация
                top_p=0.95,          # nucleus sampling
                temperature=0.8
            )[0]["generated_text"][len(x_text)+1:]
  data.append([x_text,y_text, rnn_generated, gpt_generated])
  # print(f'{i}\n', pretty_output(x_text,
  #                     y_text,
  #                     rnn_generated,
  #                     gpt_generated))

In [ ]:
dfc = pd.DataFrame(data, columns=['input' ,'real_output', 'rnn_generated', 'gpt_generated'])
for col in ['input' ,'real_output', 'rnn_generated', 'gpt_generated']:
    dfc[f'{col}_words_count'] = dfc[col].apply(lambda x: len(x.split()))

# Этап 5. Формулирование выводов

## 5.1 Сравнение предсказаний и мертик двух моделей

In [ ]:
df_rouge

In [ ]:
dfc

## 5.2 Выводы

- 1. Модель RNN генерирует тексты соизмеримой длины относительно истинных текстов. Модель GPT генерирует тексты на порядок длиннее и сложнее, чем требуется 
- 2. Модель RNN выдает однотипные ответы, в ней мало разнообразия. Модель GPT генерирует более разнообразные ответы
- 3. Модель RNN по метрикам rouge превосходит модель GPT. 
- 4. Модель RNN показала результат (по метрикам) лучше, чем  GPT за счет целенаправленного обучения на нужных данных
- 5. Несмотря на п.4 RNN часто выдает похожие ответы на совершенно разные входящие промты. Это говорит о ее ограниченных пределах возможностей
- 6. У GPT потенциал больше, чем у RNN т.к. видим что способен выдавать более длинные и осмысленные ответы. Для этого необходимо обучить GPT на наших данных